# Capstone Project
Copy and paste Github link into https://nbviewer.jupyter.org to better view Github .ipynb files 

## Code



In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Importing the Web Scraping libraries
# Install these into your system/environment if not downloaded
import requests
import lxml.html as lh

def get_content(url):
    #Create a handle, page, to handle the contents of the website
    page = requests.get(url)
    #Store the contents of the website under doc
    doc = lh.fromstring(page.content)
    return doc

def get_coord(zip_code):
    url = 'https://www.zip-codes.com/m/zip-code/' + zip_code + '/zip-code-' + zip_code + '.asp'
    zipDoc = get_content(url)
    tr_elements2 = zipDoc.xpath('//tr')
    lat = float(tr_elements2[11].text_content()[9:])
    long = float(tr_elements2[12].text_content()[10:])
    return (lat,long)

# The url for the wiki page
url = 'https://www.zip-codes.com/m/city/tx-austin.asp'
doc = get_content(url)

#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

#Create empty list
col=[]
i=0

# Geting Column Names
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    col.append((name,[]))

# Getting the values for each columns
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 4, the //tr data is not from our table 
    if len(T)!=4:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)
df = df[['ZIP Code']]
df.columns = ['zipCode']

df[['Latitude','Longitude']] = pd.DataFrame(df.zipCode.apply(get_coord).tolist())
df.shape

(73, 3)

In [2]:
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

In [3]:
address = 'Austin, TX'

geolocator = Nominatim(user_agent="explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Austin are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Austin are 30.2711286, -97.7436995.


In [4]:
# create map of Austin using latitude and longitude values
map_austin = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map. On my PC .CircleMarker() does not have the parameter 'parse_html' so I removed it.
# I also changed the dataframe name to dfComplete
for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df['zipCode']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_austin)  
    
map_austin

In [5]:
CLIENT_ID = 'UPX3DQWNOVURCU4JGBGCMOQU023OTZJIYE3QWLKF53XUFML4' # your Foursquare ID
CLIENT_SECRET = 'JWPBHGHKGM1CI3WFWDFHOQYVG0WZI2ZSHCM4GT5QZU1YMLA4' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: UPX3DQWNOVURCU4JGBGCMOQU023OTZJIYE3QWLKF53XUFML4
CLIENT_SECRET:JWPBHGHKGM1CI3WFWDFHOQYVG0WZI2ZSHCM4GT5QZU1YMLA4


In [6]:
postal_latitude = df.loc[0, 'Latitude'] # Postal Code latitude value
postal_longitude = df.loc[0, 'Longitude'] # Postal Code longitude value

postal_name = df.loc[0, 'zipCode'] # Postal Code

print('Latitude and longitude values of {} are {}, {}.'.format(postal_name, 
                                                               postal_latitude, 
                                                               postal_longitude))
LIMIT = 30
radius = 804.672# meters; .5 miles
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    postal_latitude, 
    postal_longitude, 
    radius, 
    LIMIT)

Latitude and longitude values of 73301 are 30.219702, -97.74726.


In [7]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [8]:
results = requests.get(url).json()

In [9]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]


print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

27 venues were returned by Foursquare.


In [10]:
def getNearbyVenues(names, latitudes, longitudes, radius=804.672, limit = 50):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['zipCode', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [11]:
austinV = getNearbyVenues(names=df['zipCode'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

73301
73344
78701
78702
78703
78704
78705
78708
78709
78710
78711
78712
78713
78714
78715
78716
78717
78718
78719
78720
78721
78722
78723
78724
78725
78726
78727
78728
78729
78730
78731
78732
78733
78734
78735
78736
78737
78738
78739
78741
78742
78744
78745
78746
78747
78748
78749
78750
78751
78752
78753
78754
78755
78756
78757
78758
78759
78760
78761
78762
78763
78764
78765
78766
78767
78768
78772
78773
78774
78778
78779
78783
78799


In [12]:
print(austinV.shape)

(2054, 7)


In [13]:
austinV.groupby('zipCode').count()

,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
zipCode,,,,,,
73301,27,27,27,27,27,27
73344,50,50,50,50,50,50
78701,50,50,50,50,50,50
78702,47,47,47,47,47,47
78703,13,13,13,13,13,13
78704,25,25,25,25,25,25
78705,50,50,50,50,50,50
78708,50,50,50,50,50,50
78709,50,50,50,50,50,50


In [14]:
print('There are {} uniques categories.'.format(len(austinV['Venue Category'].unique())))

There are 205 uniques categories.


In [15]:
# one hot encoding
austinOneHot = pd.get_dummies(austinV[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
austinOneHot['zipCode'] = austinV['zipCode'] 

# move neighborhood column to the first column
fixed_columns = [austinOneHot.columns[-1]] + list(austinOneHot.columns[:-1])
austinOneHot = austinOneHot[fixed_columns]

In [16]:
austinG = austinOneHot.groupby('zipCode').mean().reset_index()

In [17]:
num_top_venues = 15

for zipCode in austinG['zipCode']:
    print("----"+zipCode+"----")
    temp = austinG[austinG['zipCode'] == zipCode].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----73301----
                     venue  freq
0                    Hotel  0.19
1       Mexican Restaurant  0.11
2     Fast Food Restaurant  0.07
3                     Café  0.04
4             Liquor Store  0.04
5              Gas Station  0.04
6                     Park  0.04
7              Coffee Shop  0.04
8            Grocery Store  0.04
9               Food Truck  0.04
10          Sandwich Place  0.04
11  Furniture / Home Store  0.04
12                Pharmacy  0.04
13             Video Store  0.04
14           Auto Workshop  0.04


----73344----
                 venue  freq
0                Hotel  0.10
1          Coffee Shop  0.08
2   Italian Restaurant  0.04
3                  Bar  0.04
4           Steakhouse  0.04
5            Speakeasy  0.04
6         Cocktail Bar  0.04
7   Seafood Restaurant  0.04
8           Restaurant  0.04
9         Burger Joint  0.04
10       Grocery Store  0.04
11              Lounge  0.04
12          Art Museum  0.04
13           Juice Bar  0.04
14     

                       venue  freq
0                 Food Truck  0.17
1       Fast Food Restaurant  0.07
2               Cupcake Shop  0.03
3   Mediterranean Restaurant  0.03
4           Ramen Restaurant  0.03
5        Rental Car Location  0.03
6                Pizza Place  0.03
7             Sandwich Place  0.03
8                       Park  0.03
9                Coffee Shop  0.03
10        Chinese Restaurant  0.03
11                Taco Place  0.03
12               Golf Course  0.03
13                   Theater  0.03
14        Mexican Restaurant  0.03


----78723----
                      venue  freq
0            Cosmetics Shop  0.08
1   Health & Beauty Service  0.08
2                Food Truck  0.08
3         Mobile Phone Shop  0.08
4      Fast Food Restaurant  0.08
5                      Park  0.08
6             Grocery Store  0.08
7      Gym / Fitness Center  0.08
8                Donut Shop  0.08
9            Baseball Field  0.08
10         Business Service  0.08
11              

                            venue  freq
0                Business Service   0.5
1                            Farm   0.5
2               Accessories Store   0.0
3                Ramen Restaurant   0.0
4                     Music Venue   0.0
5         New American Restaurant   0.0
6                    Night Market   0.0
7                       Nightclub   0.0
8                    Optical Shop   0.0
9               Other Repair Shop   0.0
10              Outdoor Sculpture   0.0
11                   Outlet Store   0.0
12  Paper / Office Supplies Store   0.0
13                           Park   0.0
14          Performing Arts Venue   0.0


----78744----
                            venue  freq
0                           Trail  0.25
1                    Soccer Field  0.25
2                            Park  0.25
3                         Dog Run  0.25
4   Paper / Office Supplies Store  0.00
5                          Museum  0.00
6                     Music Venue  0.00
7         New American R

14           Gift Shop  0.02


----78762----
                 venue  freq
0                Hotel  0.12
1         Cocktail Bar  0.06
2          Coffee Shop  0.06
3   Italian Restaurant  0.04
4        Movie Theater  0.04
5            Speakeasy  0.04
6   Seafood Restaurant  0.04
7           Restaurant  0.04
8                  Bar  0.04
9         Burger Joint  0.04
10       Grocery Store  0.04
11              Lounge  0.04
12          Art Museum  0.04
13           Juice Bar  0.04
14           Gift Shop  0.02


----78763----
                 venue  freq
0                Hotel  0.12
1         Cocktail Bar  0.06
2          Coffee Shop  0.06
3   Italian Restaurant  0.04
4        Movie Theater  0.04
5            Speakeasy  0.04
6   Seafood Restaurant  0.04
7           Restaurant  0.04
8                  Bar  0.04
9         Burger Joint  0.04
10       Grocery Store  0.04
11              Lounge  0.04
12          Art Museum  0.04
13           Juice Bar  0.04
14           Gift Shop  0.02


----78764

In [18]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [19]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['zipCode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
zipCodeS = pd.DataFrame(columns=columns)
zipCodeS['zipCode'] = austinG['zipCode']

for ind in np.arange(austinG.shape[0]):
    zipCodeS.iloc[ind, 1:] = return_most_common_venues(austinG.iloc[ind, :], num_top_venues)

zipCodeS.head()

,zipCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,73301,Hotel,Mexican Restaurant,Fast Food Restaurant,Pharmacy,Coffee Shop
1,73344,Hotel,Coffee Shop,Burger Joint,Grocery Store,Bar
2,78701,Cocktail Bar,Hotel,Movie Theater,Bar,Speakeasy
3,78702,Coffee Shop,Mexican Restaurant,Pub,Fast Food Restaurant,Park
4,78703,Harbor / Marina,Vegetarian / Vegan Restaurant,Park,Shipping Store,Shoe Store


In [20]:
# set number of clusters
kclusters = 10

austinC = austinG.drop('zipCode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(austinC)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[:] 

array([1, 1, 1, 9, 9, 9, 9, 1, 1, 9, 1, 9, 1, 1, 1, 1, 9, 1, 1, 9, 9, 9,
       6, 7, 9, 9, 9, 9, 9, 9, 9, 1, 8, 5, 3, 9, 9, 0, 9, 2, 9, 0, 4, 9,
       1, 9, 1, 9, 9, 9, 9, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1])

In [21]:
zipCodeS.insert(0, 'Cluster Labels', kmeans.labels_)

austinM = df

austinM = austinM.join(zipCodeS.set_index('zipCode'), on='zipCode')
austinM['Cluster Labels'] = austinM['Cluster Labels'].fillna(0.0).astype(int)
austinM.head() # check the last columns!

,zipCode,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,73301,30.219702,-97.747260,1,Hotel,Mexican Restaurant,Fast Food Restaurant,Pharmacy,Coffee Shop
1,73344,30.266900,-97.742900,1,Hotel,Coffee Shop,Burger Joint,Grocery Store,Bar
2,78701,30.267200,-97.742306,1,Cocktail Bar,Hotel,Movie Theater,Bar,Speakeasy
3,78702,30.263915,-97.713660,9,Coffee Shop,Mexican Restaurant,Pub,Fast Food Restaurant,Park
4,78703,30.289730,-97.766479,9,Harbor / Marina,Vegetarian / Vegan Restaurant,Park,Shipping Store,Shoe Store


In [22]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(austinM['Latitude'], austinM['Longitude'], austinM['zipCode'], austinM['Cluster Labels'].astype(int)):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [23]:
austinM.loc[austinM['Cluster Labels'] == 0, austinM.columns[[0] + list(range(4, austinM.shape[1]))]]

,zipCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
18,78719,NaN,NaN,NaN,NaN,NaN
29,78730,NaN,NaN,NaN,NaN,NaN
32,78733,NaN,NaN,NaN,NaN,NaN
38,78739,NaN,NaN,NaN,NaN,NaN
41,78744,Soccer Field,Park,Dog Run,Trail,Zoo
44,78747,NaN,NaN,NaN,NaN,NaN
46,78749,Park,Pool,Fast Food Restaurant,Zoo,Dive Bar
51,78754,NaN,NaN,NaN,NaN,NaN


In [24]:
austinM.loc[austinM['Cluster Labels'] == 1, austinM.columns[[0] + list(range(4, austinM.shape[1]))]]

,zipCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,73301,Hotel,Mexican Restaurant,Fast Food Restaurant,Pharmacy,Coffee Shop
1,73344,Hotel,Coffee Shop,Burger Joint,Grocery Store,Bar
2,78701,Cocktail Bar,Hotel,Movie Theater,Bar,Speakeasy
7,78708,Hotel,Cocktail Bar,Seafood Restaurant,Coffee Shop,Italian Restaurant
8,78709,Hotel,Cocktail Bar,Seafood Restaurant,Coffee Shop,Italian Restaurant
10,78711,Hotel,Cocktail Bar,Seafood Restaurant,Coffee Shop,Italian Restaurant
12,78713,Hotel,Cocktail Bar,Seafood Restaurant,Coffee Shop,Italian Restaurant
13,78714,Hotel,Cocktail Bar,Seafood Restaurant,Coffee Shop,Italian Restaurant
14,78715,Hotel,Cocktail Bar,Seafood Restaurant,Coffee Shop,Italian Restaurant
15,78716,Hotel,Cocktail Bar,Seafood Restaurant,Coffee Shop,Italian Restaurant


In [25]:
austinM.loc[austinM['Cluster Labels'] == 2, austinM.columns[[0] + list(range(4, austinM.shape[1]))]]

,zipCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
43,78746,Insurance Office,Zoo,Doctor's Office,Food,Field


In [26]:
austinM.loc[austinM['Cluster Labels'] == 3, austinM.columns[[0] + list(range(4, austinM.shape[1]))]]

,zipCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
37,78738,Harbor / Marina,Zoo,Dive Bar,Field,Fast Food Restaurant


In [27]:
austinM.loc[austinM['Cluster Labels'] == 4, austinM.columns[[0] + list(range(4, austinM.shape[1]))]]

,zipCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
47,78750,Photography Studio,Zoo,Dive Bar,Field,Fast Food Restaurant


In [28]:
austinM.loc[austinM['Cluster Labels'] == 5, austinM.columns[[0] + list(range(4, austinM.shape[1]))]]

,zipCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
36,78737,Pool,Zoo,Dive Bar,Field,Fast Food Restaurant


In [29]:
austinM.loc[austinM['Cluster Labels'] == 6, austinM.columns[[0] + list(range(4, austinM.shape[1]))]]

,zipCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
23,78724,General Entertainment,Athletics & Sports,Zoo,Doctor's Office,Field


In [30]:
austinM.loc[austinM['Cluster Labels'] == 7, austinM.columns[[0] + list(range(4, austinM.shape[1]))]]

,zipCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
24,78725,Toll Booth,Zoo,Food & Drink Shop,Field,Fast Food Restaurant


In [31]:
austinM.loc[austinM['Cluster Labels'] == 8, austinM.columns[[0] + list(range(4, austinM.shape[1]))]]

,zipCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
35,78736,Zoo,Dive Bar,Field,Fast Food Restaurant,Farm


In [32]:
austinM.loc[austinM['Cluster Labels'] == 9, austinM.columns[[0] + list(range(4, austinM.shape[1]))]]

,zipCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
3,78702,Coffee Shop,Mexican Restaurant,Pub,Fast Food Restaurant,Park
4,78703,Harbor / Marina,Vegetarian / Vegan Restaurant,Park,Shipping Store,Shoe Store
5,78704,Liquor Store,Park,Spa,Coffee Shop,Wine Bar
6,78705,Food Truck,Burger Joint,Sandwich Place,Mexican Restaurant,Taco Place
9,78710,Theme Park,Brewery,Hotel,Convenience Store,Asian Restaurant
11,78712,Coffee Shop,Sandwich Place,Pool,History Museum,Fast Food Restaurant
16,78717,American Restaurant,Gym / Fitness Center,Donut Shop,Liquor Store,Pharmacy
20,78721,Home Service,Trail,Circus School,Night Market,Music Venue
21,78722,Food Truck,Fast Food Restaurant,Cupcake Shop,Diner,Mexican Restaurant
22,78723,Food Truck,Fast Food Restaurant,Cosmetics Shop,Health & Beauty Service,Mobile Phone Shop
